In [37]:
%reload_ext autoreload
%autoreload 2
import sys
sys.path.insert(1, '/p/project/chhb19/gaertner2/lkf_detection/functions/')
from statistics_functions import *
sys.path.insert(1, '/p/project/chhb19/gaertner2/lkf_tools/lkf_tools/')
from dataset import *
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [38]:
path = '/p/project/chhb19/gaertner2/data/awicm_cvmix/'
path_nc = '/p/scratch/chhb19/gaertner2/interpolated_fesom_output/'
path_ds = path + 'datasets/'
path_imgs = path + 'imgs/'

# Calculate LKF frequency

For this, just the detection has to be done. This is independet of other metrics and notebooks. It is only saved to the LKF_map.npy file.

In [49]:
years = [2013, 2015, 2017, 2019, 2020,
         2093, 2094, 2095, 2096, 2097, 2100]

In [50]:
datasets = [np.load(path_ds + f'ds_{year}.npy', allow_pickle=True)[0] for year in years]

In [51]:
res_km = get_res_km(datasets[0])
mask = np.where(np.isnan(datasets[0].calc_eps(0)[1]), np.nan, 1)

coarse_grid_box_size_km = 25

In [52]:
%%time
LKF_map = []
for lkf_data in datasets:
    LKF_map_ds = []
    for i in range(365):
        LKF_map_ds += coarse_graining(lkf_data.finetuning(i=i, dog_thres=0.1, plot=False), coarse_grid_box_size_km, res_km),
    LKF_map += LKF_map_ds,
    print('.')

.
.
.
.
.
.
.
.
.
.
.
CPU times: user 22min 52s, sys: 16.7 s, total: 23min 9s
Wall time: 23min 27s


In [53]:
days = np.append([0], xticks)

In [54]:
LKF_map_monthly = []
for LKF_map_dataset in LKF_map:
    
    LKF_map_monthly_y = []
    for startday, endday in zip(days, np.roll(days,-1)):
        
        tmp = []
        for i in range(startday, endday):
            
            tmp += LKF_map_dataset[i],
        LKF_map_monthly_y += np.sum(tmp,axis=0)/(endday-startday)*mask,
        
    del LKF_map_monthly_y[-1] # this is to delete the day=365 to day=0 element
    LKF_map_monthly += LKF_map_monthly_y,

In [55]:
np.shape(LKF_map_monthly)

(11, 12, 752, 939)

In [56]:
np.save('/p/scratch/chhb19/gaertner2/LKF_map_monthly_d01', LKF_map_monthly)